Question -6 . Collaborative Filtering (Distributed Stochastic Gradient Descent for Matrix Factorization)

*Group Members*
1. 1741031: Foram Vadher
2. 1741034: Krinali Shah
3. 1741074: Arpit Patel
4. 1741079: Vaishwi Patel

#**Intiatial steps**

**Intializing Libraries**

In [0]:
import pandas as pd
import numpy as np
# for information how many cpu in the processor
import multiprocessing as mp

print(mp.cpu_count())

2


**Initializing Variables**

In [0]:
# R is a rating matrix R_n*m where n total number of user , m is total number of items
# U is user matrix U_k*n k is feature of item
# V is item matrix V_k*m k is feature of item

n = 12 # number of user
m = 60 # number of item
k = 10 #number of item feature

R = np.random.randint(5, size=(n, m))
U = np.random.randint(5, size=(k, n))
V = np.random.randint(5, size=(k, m))
d = 3  #number of worker

#**All necessary Functions**

Create a block for U, V and R

In [0]:
def block_R(d,m,n,R):
  row_of_block = n//d
  col_of_block = m//d

  R_new = np.zeros(d*d*(n//d)*(m//d)).reshape((d,d,n//d,m//d))
  r_start = 0
  r_end   = row_of_block

  for i in range(d):
    c_start = 0
    c_end = col_of_block
    for j in range(d):
       
      R_new[i][j] = R[r_start:r_end,c_start:c_end]

      c_start = c_end
      c_end = c_start + col_of_block

    r_start = r_end
    r_end   = r_start + row_of_block

  return R_new

def parts_U_V(d,k,n,m,U,V):
  
  row_of_U = k
  col_of_U = n//d
  row_of_V =k
  col_of_V = m//d

  U_new = np.zeros(d*k*(n//d)).reshape((d,k,n//d))
  V_new = np.zeros(d*k*(m//d)).reshape((d,k,m//d))
  # create parts of U and V
  first_U = 0
  last_U = col_of_U

  first_V = 0
  last_V = col_of_V

  print(row_of_U,col_of_U,row_of_V,col_of_V)

  for i in range(d):
    #print(first_U,last_U)
    U_new[i] = U[:,first_U:last_U] 
    V_new[i] = V[:,first_V:last_V] 

    first_U = last_U
    last_U = first_U + col_of_U

    first_V = last_V
    last_V = first_V + col_of_V

  return U_new,V_new


Permutations

In [0]:

def permutation_for_block(d):
  data = ''
  for i in range(d):
    data = data+str(i+1)
  #print("Data : ",data,type(data))
  #data='12'
  per = permutation(data)
  i=0
  for p in per:
    per[i] = [int(i) for i in p] 
    #print (per[i])
    i=i+1

  final =select_permutation(per,d)
  print(final)
  return final

# Selecte permutation where no columns and rows are match and cover all the block of R
def select_permutation(per,d):
  ini = per[0]
  cols =d
  rows=d
  final = [[]] 
  final[0]=ini
  eva = [] ## for evalute whetere that is taken or nor
  temp = [0]*d

  for i in range(d):
    eva.append(int(str(i+1)+str(ini[i])))

  print(eva[0])
  row_count=1
  eva_cnt = d

  for i in range(1,len(per)):
    #print("In for")
    for k in range(d):
      temp[k]=int(str(k+1)+str(per[i][k]))
    
    #print("temp:",temp)
    count =0
    for j in range(d):
      if temp[j] in eva:
        count =count+1

    if count == 0:
      #print("In add")
      final.append(per[i])
      row_count = row_count+1

      for k in range(d):
        eva.append(int(str(k+1)+str(per[i][k])))
      
      eva_cnt = eva_cnt+d

  return final


# Python function to print permutations of a given list 
def permutation(lst): 

  # If lst is empty then there are no permutations 
  if len(lst) == 0: 
    return [] 

  # If there is only one element in lst then, only 
  # one permuatation is possible 
  if len(lst) == 1: 
    return [lst] 

  # Find the permutations for lst if there are 
  # more than 1 characters 

  l = [] # empty list that will store current permutation 

  # Iterate the input(lst) and calculate the permutation 
  for i in range(len(lst)): 
    m = lst[i] 

    # Extract lst[i] or m from the list. remLst is 
    # remaining list 
    remLst = lst[:i] + lst[i+1:] 

    # Generating all permutations where m is first 
    # element 
    for p in permutation(remLst): 
      l.append([m] + [p])
    
  return l 



**Worker**

In [0]:
def worker(i,B_pq,U_p,V_q,n,m):

  eta=0.1  #learning rate 
  lamda =0.1 #regularization parameter
  #here i=row ,j=col
  [i,j]=size(B_pq) #what is i,j see ppt of lecture

  for row in range(i):
    for col in range(j):
      U[:,i] = U[:,i] - 2*eta*((np.sum(np.multiply(U[:,i].T,V[:,j]))-R[i,j])*V[:,j]+(lamda/m)*U[:,i])
      V[:,j] = V[:,j] - 2*eta*((np.sum(np.multiply(U[:,i].T,V[:,j]))-R[i,j])*U[:,i]+(lamda/n)*V[:,j])

  return (i,U_p,V_q)
  

**Master**

In [0]:

 # Step 2: Define callback function to collect the output in `results`
def collect_result(result):
	global results
	results.append(result)

def master(R,U,V,d,m,n,k):
  pi_p = permutation_for_block(d)

  results = []
  
  for i in range(d):
    pool = mp.Pool(d)
    for p in range(d):
      print(p,pi_p[i][p]-1)
      pool.apply_async(worker, args=(p,R[p][pi_p[i][p]-1],U[p],V[pi_p[i][p]-1],n,m),callback=collect_result)
    # Step 4: Close Pool and let all the processes complete    
    pool.close()
    pool.join()  # postpones the execution of next line of code until all processes in the queue are done.
    for j,u,v in results:
      U[j] = u
      V[pi_p[i][j]-1] = v

  return (U,V)

#**Main Function**

In [0]:
if __name__=='__main__':
  n = 12 # number of user
  m = 60 # number of item
  k = 10 #number of item feature

  R = np.random.randint(5, size=(n, m))
  U = np.random.randint(2, size=(k, n))
  V = np.random.randint(2, size=(k, m))
  d = 2  #number of worker

  [U_new , V_new]=parts_U_V(d,k,n,m,U,V)
  R_new = block_R(d,m,n,R)


  print("Shape of U before blocking",U.shape)
  print("Shape of V before blocking",V.shape)
  print("Shape of R before blocking",R.shape)
  print("Shape of U After blocking",U_new.shape)
  print("Shape of V After blocking",V_new.shape)
  print("Shape of R After blocking",R_new.shape)


  [U_p,V_q] = master(R_new,U_new,V_new,d,m,n,k)

  print("U_p = ",U_p[1])
  print("V_q = ",V_q[1])

10 6 10 30
Shape of U before blocking (10, 12)
Shape of U before blocking (10, 60)
Shape of U before blocking (12, 60)
Shape of U After blocking (2, 10, 6)
Shape of V After blocking (2, 10, 30)
Shape of R After blocking (2, 2, 6, 30)
11
[[1, 2], [2, 1]]
0 0
1 1
0 1
1 0
U_p =  [[1. 1. 0. 0. 0. 1.]
 [1. 0. 1. 0. 0. 1.]
 [1. 1. 1. 1. 0. 1.]
 [0. 1. 1. 0. 1. 1.]
 [1. 1. 1. 1. 1. 1.]
 [0. 0. 0. 0. 0. 1.]
 [0. 1. 0. 0. 0. 1.]
 [1. 0. 1. 0. 0. 0.]
 [1. 0. 0. 1. 1. 1.]
 [0. 0. 1. 0. 1. 0.]]
V_q =  [[1. 1. 0. 0. 1. 1. 1. 0. 0. 1. 0. 1. 1. 1. 1. 0. 0. 1. 1. 1. 1. 0. 1. 0.
  0. 0. 1. 0. 1. 0.]
 [1. 0. 1. 1. 1. 0. 1. 1. 1. 0. 0. 0. 1. 1. 1. 0. 1. 1. 1. 0. 0. 0. 1. 1.
  0. 1. 0. 1. 1. 1.]
 [0. 0. 1. 0. 0. 0. 0. 1. 1. 1. 0. 0. 1. 1. 0. 1. 1. 0. 0. 1. 1. 1. 1. 1.
  1. 0. 0. 0. 1. 0.]
 [1. 0. 1. 1. 0. 1. 0. 0. 0. 0. 1. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 1.
  1. 0. 1. 1. 1. 0.]
 [1. 1. 0. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 0. 0. 1. 1. 1. 0. 0.
  1. 0. 0. 1. 1. 0.]
 [0. 0. 1. 0. 0. 1. 1. 1. 1. 0

# Parralel (Multiprocessing Testing)

**Prepare a data**

In [0]:

# take info about how many cpu in this computer
import multiprocessing as mp
print("Number of processors: ", mp.cpu_count())


#here this is example of asynchronization because we need this in AOBD

# Parallel processing with Pool.apply_async()

import multiprocessing as mp
pool = mp.Pool(mp.cpu_count())

results = []



###define a data
import numpy as np
from time import time

# Prepare data
np.random.RandomState(100)
arr = np.random.randint(0, 10, size=[200000, 5])
data = arr.tolist()
data[:5]


**Parallelizing using Pool.map()**

In [0]:


def howmany_within_range_rowonly(row, minimum=4, maximum=8):
  count = 0
  for n in row:
    if minimum <= n <= maximum:
      count = count + 1
  return count

# Step 2: Define callback function to collect the output in `results`
def collect_result(result):
  global results
  results.append(result)


# Step 3: Use loop to parallelize
results = pool.map(howmany_within_range_rowonly, [row for row in data])

# Step 4: Close Pool and let all the processes complete
print("before close")
pool.close()
print("After close")
pool.join()  # postpones the execution of next line of code until all processes in the queue are done.
print("After join")



print(results[:10])
#> [3, 1, 4, 4, 4, 2, 1, 1, 3, 3]




Number of processors:  2
before close
After close
After join
[2, 3, 3, 0, 4, 3, 3, 3, 1, 1]


**Parallelizing with Pool.apply_async()**

In [0]:
# Parallel processing with Pool.apply_async()

import multiprocessing as mp
pool = mp.Pool(mp.cpu_count())

results = []

# Step 1: Redefine, to accept `i`, the iteration number
def howmany_within_range2(i, row, minimum, maximum):
  count = 0
  for n in row:
    if minimum <= n <= maximum:
      count = count + 1
  return (i, count)


# Step 2: Define callback function to collect the output in `results`
def collect_result(result):
  global results
  results.append(result)


# Step 3: Use loop to parallelize
for i, row in enumerate(data):
  pool.apply_async(howmany_within_range2, args=(i, row, 4, 8), callback=collect_result)

# Step 4: Close Pool and let all the processes complete    
pool.close()
pool.join()  # postpones the execution of next line of code until all processes in the queue are done.

# Step 5: Sort results [OPTIONAL]
results.sort(key=lambda x: x[0])
results_final = [r for i, r in results]

print(results_final[:10])
#> [3, 1, 4, 4, 4, 2, 1, 1, 3, 3]


[2, 3, 3, 0, 4, 3, 3, 3, 1, 1]
